In [634]:
import pandas as pd

comments=pd.read_csv('data/comments.csv')
stats=pd.read_csv('data/videos-stats.csv')
comments.head()


,Unnamed: 0,Video ID,Comment,Likes,Sentiment
0,0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0
1,1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0
2,2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0
3,3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0
4,4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0


In [635]:
# delete invalid col
comments=comments.drop(columns=['Unnamed: 0'],axis=1)
stats=stats.drop(columns=['Unnamed: 0'],axis=1)

# delete nan value
stats=stats.dropna()
comments=comments.dropna()

In [636]:
# caculate pop 
stats["pop"]=stats.loc[:,["Likes","Comments","Views"]].apply(lambda x:x["Likes"]*0.3+x["Comments"]*0.6+x['Views'],axis=1)

stats.head()
# comments likes views scale diff,so they need to be normalizeed


,Title,Video ID,Published At,Keyword,Likes,Comments,Views,pop
0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,2022-08-23,tech,3407.0,672.0,135612.0,137037.3
1,The most EXPENSIVE thing I own.,b3x28s61q3c,2022-08-24,tech,76779.0,4306.0,1758063.0,1783680.3
2,My New House Gaming Setup is SICK!,4mgePWWCAmA,2022-08-23,tech,63825.0,3338.0,1564007.0,1585157.3
3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,2022-08-23,tech,71566.0,1426.0,922918.0,945243.4
4,Best Back to School Tech 2022!,ErMwWXQxHp0,2022-08-08,tech,96513.0,5155.0,1855644.0,1887690.9


1. 下面生成的表 可以用作**时序分析**

In [637]:
# nomalization
# 在Python的sklearn包中提供了StandardScaler 和MinMaxScaler 可以很方便的对数据进行规范化处理
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing as sp

video_likes=stats.loc[:,'Likes']
video_comments=stats.loc[:,'Comments']
video_views=stats.loc[:,'Views']

standard = sp.MinMaxScaler()
stats.loc[:,'Likes'] = standard.fit_transform(video_likes.values.reshape(-1,1))
stats.loc[:,'Comments']=standard.fit_transform(video_comments.values.reshape(-1,1))
stats.loc[:,'Views']=standard.fit_transform(video_views.values.reshape(-1,1))

stats["pop"]=stats.loc[:,["Likes","Comments","Views"]].apply(lambda x:x["Likes"]*0.3+x["Comments"]*0.6+x['Views'],axis=1)
stats.to_csv('data/pop_stats.csv')

# pop 的值已经算出来，整合的数据集在目录 data/data/pop_stats.csv 下，可以用来做 时序分析

2. 接下来是 **生成 tag** 和 为了做 **clustering** 的准备工作

In [639]:
# 分词
# 使用了 stopwords，词元化，标点，还有自定义的stopwords
# 使用了 nltk 包里的 word_tokenize 对 sentence 进行 tokenization
# 使用 WordNetLemmatizer 对 tokens 进行词元化分析和筛选

# 生成有 tokens 列的表,如下的输出，但此时并没有通过 id 进行聚合，也还没提取出 关键词 ，因此这个表还不能用于 clustering
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# stopwords
new_stopwords = ["video", "s", "to", "on", "daily"]
stpwrd = nltk.corpus.stopwords.words('english')
stpwrd.extend(new_stopwords)
stpwrd.extend(string.punctuation)


def generate_tag(comments):
    word_tokens=word_tokenize(comments)
    lemmatized_output =[wordnet_lemmatizer.lemmatize(w) for w in word_tokens]
    filter_sen=[w for w in lemmatized_output if not w.lower() in stpwrd]

    return filter_sen

comment_words = ''


comments['Comment_tag']=comments['Comment'].apply(generate_tag)

comments['Comment_tag']=comments['Comment_tag'].apply(str)

comments[0:10]

[nltk_data] Downloading package stopwords to /home/qlchen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/qlchen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/qlchen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/qlchen/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Video ID,Comment,Likes,Sentiment,Comment_tag
0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0,"['Let', ""'s"", 'forget', 'Apple', 'Pay', '2014'..."
1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0,"['NZ', '50', 'retailer', '’', 'even', 'contact..."
2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0,"['forever', 'acknowledge', 'channel', 'help', ..."
3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0,"['Whenever', 'go', 'place', '’', 'take', 'Appl..."
4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0,"['Apple', 'Pay', 'convenient', 'secure', 'easy..."
5,wAZZ-UWGVHI,We’ve been hounding my bank to adopt Apple pay...,8.0,1.0,"['’', 'hounding', 'bank', 'adopt', 'Apple', 'p..."
6,wAZZ-UWGVHI,We only got Apple Pay in South Africa in 2020/...,29.0,2.0,"['got', 'Apple', 'Pay', 'South', 'Africa', '20..."
7,wAZZ-UWGVHI,"For now, I need both Apple Pay and the physica...",7.0,1.0,"['need', 'Apple', 'Pay', 'physical', 'credit',..."
8,wAZZ-UWGVHI,"In the United States, we have an abundance of ...",2.0,2.0,"['United', 'States', 'abundance', 'retailer', ..."
9,wAZZ-UWGVHI,"In Cambodia, we have a universal QR code syste...",28.0,1.0,"['Cambodia', 'universal', 'QR', 'code', 'syste..."


3. 根据 ID 聚合数据，这里保存的 csv 文件可以用来做 clustering，这是没经过 keyword 筛选的所有 tokens，当然也可以选择 第四点生成的 keywords 文件做，base on 你自己选择

In [645]:
data1=comments.groupby('Video ID')['Comment_tag'].apply(sum)
data1.to_csv('data/comments_tag.csv')

df_comment=pd.DataFrame(data1)

df_comment[0:10]

,Comment_tag
Video ID,
--ZI0dSbbNU,"['’', 'happy', 'eating', 'see', 'u', 'sufferin..."
--hxd1CrOqg,"['’', 'really', 'unfortunate', 'intended', 'ta..."
--ixiTypG8g,"['biggest', 'problem', 'college', 'charging', ..."
-64r1hcxtV4,"['8:20-', '10:00', '.....', 'thought', 'happen..."
-6IgkG5yZfo,"['Hope', 'make', 'life', 'easier', '..', '00:4..."
-7hzaGya86g,"['Yes', 'Dinner', 'dancing', 'Lotte', 'Mart', ..."
-8TnsjDRXUE,"['ha', 'sponsored', 'Onnit', 'Visit', 'http', ..."
-9hjdvULDyc,"['Hope', 'everyone', 'get', 'perfect', '1600',..."
-Cr69sGnrk8,"['Let', ""'s"", 'appreciate', 'doe', 'far', ""'s""..."


4. 生成每个 video 的关键词 使用 summa 的 keywords 方法直接提取，生成的表，可以直接用于 clustering，这里生成的 keyword 才是代表每个 video 的 tag,数据保存在 data/comment_keyword_true_tag.csv 下，可以直接 read 做 clustering

In [647]:
from summa import keywords

def generate_key(comm):

    TR_keywords = keywords.keywords(comm, scores=True) 
    com_key=[]
    for x,y in TR_keywords:
        com_key.append(x)
    return com_key



df_comment['Comment_keyword']=df_comment['Comment_tag'].apply(generate_key)
# for i range(len(df_comment)):
#     df_comment['Comment'][i]=df_comment['Comment'][i]


In [649]:

df_comment.to_csv('data/comment_keyword_true_tag.csv')
df_comment['Comment_keyword'][0:10]

Video ID
--ZI0dSbbNU    [eating, eat, eats, fry, feel, smile, man, suf...
--hxd1CrOqg    [russia, til, n, wo, realize, ukraine, analyza...
--ixiTypG8g    [college, colleges, tuition, regulated, regula...
-64r1hcxtV4    [life, inspiration, inspiring, inspire, health...
-6IgkG5yZfo    [thankful, thank, thanks, time, help, helped, ...
-7hzaGya86g    [good, like, mikey, dinner, food, eating, dump...
-8TnsjDRXUE    [kid, amazing, know, knowing, guy, time, work,...
-9hjdvULDyc    [reading, prediction, th, nd, wa, writing, tes...
-Cr69sGnrk8    [doe, u, fan, thank, thanks, sccoby, support, ...
-D4S6TpnO44                                           [getbonus]
Name: Comment_keyword, dtype: object